In [1]:
require 'torch';
require 'nn';

START = "$START$"
FINISH = "$END$"
FILE_PATH = "dataset/language_model.data"
WORD_VEC_SIZE = 25
WINDOW_SIZE = 5

In [2]:
-- initialize Random Word Vec
function initWordVec()
    return torch.randn(WORD_VEC_SIZE)
end

-- read and Store Data
function readFile(file_path)
    local f = io.open(file_path)
    local store = {}
    local word_dict = {}
    word_dict[START] = initWordVec()
    word_dict[FINISH] = initWordVec()
    while true do
        local l = f:read()
        if not l then break end
        local words = {}
        table.insert(words, START)
        for word in l:gmatch("%w+") do 
            table.insert(words, word)
            if not word_dict[word] then
                word_dict[word] = initWordVec()
            end
        end
        table.insert(words,FINISH)
        table.insert(store, words)
    end
    return store, word_dict
end

-- split into windows and make training data
function makeTrainingData(store, window_size, word_dict)
    local train_data = {}
    for idx, line in ipairs(store) do
        local size = table.getn(line)
        local no_of_windows = math.max( 0, size - window_size) + 1

        for widx = 1, no_of_windows do
            local training_inst = {}
            local data = {}
            local word_idx = widx

            --Add to data the words for the window
            while table.getn(data) < window_size and word_idx < size do
                table.insert(data, line[word_idx])
                word_idx = word_idx + 1
            end
                
            --Align to Window Size
            while word_idx >= size and table.getn(data) < window_size do
                table.insert(data, FINISH)
                word_idx = word_idx + 1
            end
            -- Get WordVector From Dictionary
            local word_vec_data = word_dict[data[1]]

            for i = 2, table.getn(data) do
                word_vec_data = torch.cat(word_vec_data, word_dict[data[i]])
            end
            
            training_inst.data = word_vec_data
            training_inst.label = 1
            table.insert(train_data, training_inst)
        end   
    end
    return train_data
end

In [3]:
function construct_nn(windows_size, word_vec_size, hidden_layer_nodes)
    -- Add NN Layers
    local net = nn.Sequential()
    net:add(nn.Linear(window_size * word_vec_size, hidden_layer_nodes))
    net:add(nn.Sigmoid())
    net:add(nn.Linear(hidden_layer_nodes, 1))
    net:add(nn.LogSoftMax())
    -- Define Loss Function
    local criterion = nn.ClassNLLCriterion()
    return net, criterion
end

function trainAndUpdatedWordVec(net, criterion, input, output)
    for i = 1, 20 do
        -- feed it to the neural network and the criterion
        criterion:forward(net:forward(input), real_output)

        -- train over this example in 3 steps
        -- (1) zero the accumulation of the gradients
        net:zeroGradParameters()

        -- (2) accumulate gradients
        net:backward(input, criterion:backward(net.output, output))
        
        print(net.gradInput)
        
        -- (3) update parameters with a 0.01 learning rate
        net:updateParameters(0.01)
    end
    return data
end

In [6]:
store, word_dict = readFile(FILE_PATH)
train_data = makeTrainingData(store, WINDOW_SIZE, word_dict)
print(train_data)
--net, crit = construct_nn(WINDOWS_SIZE, WORD_VEC_SIZE, 50)
--for data, label in ipairs(train_data) do
    --data = trainAndUpdatedWordVec(net, crit, data, label)
--end

{
  1 : 
    {
      data : DoubleTensor - size: 125
      label : 1
    }
  2 : 
    {
      data : DoubleTensor - size: 125
      label : 1
    }
  3 : 
    {
      data : DoubleTensor - size: 125
      label : 1
    }
  4 : 
    {
      data : DoubleTensor - size: 125
      label : 1
    }
  5 : 
    {
      data : DoubleTensor - size: 125
      label : 1
    }
  6 : 
    {
      data : DoubleTensor - size: 125
      label : 1
    }
  7 : 
    {
      data : DoubleTensor - size: 125
      label : 1
    }
  8 : 
    {
      data : DoubleTensor - size: 125
      label : 1
    }
  9 : 
    {
      data : DoubleTensor - size: 125
      label : 1
    }
  10 : 
    {
      data : DoubleTensor - size: 125
      

label : 1
    }
  11 : 
    {
      data : DoubleTensor - size: 125
      label : 1
    }
}
